# 3.pandas综合应用案列


In [40]:
import pandas as pd
import numpy as np

In [2]:
file_path= '2022年成熟度数据-模型.xlsx'


In [3]:
df_full = pd.read_excel(file_path,sheet_name=None)
# 。sheet_name=None 这个参数表示读取文件中的所有工作表，
# 并将每个工作表的数据以字典的形式返回，字典的键是工作表的名称，
# 值是对应工作表的数据框。

In [4]:
type(df_full)

dict

In [5]:
for i in df_full.keys():
    print(i)

香气浓度韦恩值
OAV韦恩值
香气浓度分类信息汇总
OAV分类信息汇总
OAV
香气浓度
香气浓度3重复
多酚
多酚3重复
理化指标
理化指标3重复
感官评价总分
样品信息


In [6]:
df_sample = df_full['样品信息']
df_sample.head()    # 五列五行

,样品编号,地块信息,采样日期,天数/d
0,CXZ1-0819,葡萄-22年-成熟度刘1号地-赤霞珠-1,20220819,0
1,CXZ1-0826,葡萄-22年-成熟度刘1号地-赤霞珠-2,20220826,7
2,CXZ1-0901,葡萄-22年-成熟度刘1号地-赤霞珠-3,20220901,14
3,CXZ1-0908,葡萄-22年-成熟度刘1号地-赤霞珠-4,20220908,21
4,CXZ1-0915,葡萄-22年-成熟度刘1号地-赤霞珠-5,20220915,28


#### 读取数据表，0列重命名，将number列位置作为索引转置，重置索引，\
#### 重命名， ：按照 '样品编号' 列的值进行排序。

In [7]:
df_score = df_full['感官评价总分'].rename(columns={'Unnamed: 0':'number'}).set_index('number').T.reset_index()\
.rename(columns ={'index':'样品编号'}).sort_values('样品编号')  
df_score.head()

number,样品编号,感官得分
0,CXZ1-0915,84.333333
1,CXZ1-0920,84.500000
2,CXZ1-0925,87.166667
3,CXZ1-0930,81.666667
4,CXZ1-1005,85.333333


In [8]:
df_duofen = df_full['多酚'].rename(columns={'样品编号':'number'}).set_index('number').T.reset_index()\
.rename(columns={'index':'样品编号'}).sort_values('样品编号')

df_duofen.head()

number,样品编号,3-羟基肉桂酸,4-乙烯基愈创木酚,4-乙烯基愈创木酚,阿魏酸,白藜芦醇,表儿茶酸,表没食子儿茶素,表没食子儿茶素没食子酸酯,丁香酸,...,香草酸甲酯,香草酸乙酯,香豆酸,香兰素,杨梅素,异鼠李素,异懈皮苷,原儿茶醛,原儿茶酸,原儿茶酸甲酯
0,CXZ1-0819,0.021,0.0,0.0,0.176,0.046,57.421,0.162,0.012,1.890,...,0.037,0.042,0.000,0.013,1.034,0.247,17.305,0.012,0.087,0.007
1,CXZ1-0826,0.020,0.0,0.0,0.181,0.049,67.385,0.030,0.014,1.196,...,0.013,0.038,0.000,0.014,0.424,0.190,17.977,0.012,0.066,0.008
2,CXZ1-0901,0.024,0.0,0.0,0.236,0.243,51.254,0.130,0.018,1.685,...,0.013,0.049,0.000,0.009,0.595,0.088,5.177,0.018,0.091,0.008
3,CXZ1-0908,0.018,0.0,0.0,0.033,0.105,31.046,0.205,0.026,1.190,...,0.000,0.011,0.000,0.007,0.288,0.014,6.787,0.008,0.022,0.002
4,CXZ1-0915,0.024,0.0,0.0,0.111,0.259,31.945,0.112,0.025,1.320,...,0.000,0.014,0.024,0.006,0.646,0.045,12.580,0.011,0.041,0.004


In [9]:
df_lhzb = df_full['理化指标'].rename(columns={'Unnamed: 0':'number'}).set_index('number').T.reset_index()\
.rename(columns={'index':'样品编号'}).sort_values('样品编号')
df_lhzb.head()

number,样品编号,pH,可溶性固形物,总糖,总酸,百粒重
0,CXZ1-0819,3.82,20.48,18.8,0.92,80.4
1,CXZ1-0826,3.75,21.50,19.6,0.64,82.3
2,CXZ1-0901,3.91,23.00,20.0,0.53,82.7
3,CXZ1-0908,4.09,25.20,22.5,0.48,84.6
4,CXZ1-0915,4.22,26.60,24.3,0.41,83.5


In [14]:
df_ana = pd.merge(\
pd.merge(\
pd.merge(df_sample,df_score,how='left',left_on='样品编号',right_on='样品编号'),\   # 合并成新的dataframe,，如将 df_sample 和 df_score 按照 “样品编号” 进行左连接合并，得到一个新的 DataFrame
df_lhzb,how='left',on='样品编号'),\
df_duofen,how='left',on='样品编号')

In [24]:
df_ana_model_pre = df_ana[df_ana['感官得分'].notnull()]   # 从 df_ana 中筛选出 “感官得分” 不为空值的行，创建一个新的 DataFrame 命名为 df_ana_model_pre。
df_ana_model_pre['葡萄品种'] = df_ana_model_pre['样品编号'].apply(lambda x:x.split('-')[0])    # 样品编号分割后的第一个元素作为葡萄品种值，并新增葡萄品种一列
df_ana_model_pre   # 展示这个中间结果的 DataFrame
# 首先，从 df_ana_model_pre 中删除 “样品编号”、“地块信息”、“采样日期” 和 “葡萄品种” 这几列，
# 然后选取除最后 25 列之外的所有列。
# 接着，对 df_ana_model_pre 的 “葡萄品种” 列进行独热编码（使用 pd.get_dummies），
# 并设置 drop_first=True 表示去除第一个类别以避免多重共线性，数据类型为整数。
# concat合并，删除4-..这一列
df_ana_model = pd.concat([df_ana_model_pre.drop(['样品编号','地块信息','采样日期','葡萄品种'],axis=1).iloc[:,:-25]\
                          ,pd.get_dummies(df_ana_model_pre['葡萄品种'],drop_first=True,dtype=int)],axis=1).\
                          drop(['4-乙烯基愈创木酚'],axis=1)
# 遍历 df_ana_model 的列名，将列名中的 “/” 和 “-” 替换为空字符串，从而更新所有列名。
df_ana_model.columns = [i.replace('/','').replace('-','') for i in df_ana_model.columns]
df_ana_model['感官得分'] = df_ana_model.pop('感官得分')  # excel 裁剪
df_ana_model.rename(columns={'3羟基肉桂酸':'羟基肉桂酸_3'},inplace=True)  # 重命名
df_ana_model

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_20600\2926629619.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ana_model_pre['葡萄品种'] = df_ana_model_pre['样品编号'].apply(lambda x:x.split('-')[0])    # 样品编号分割后的第一个元素作为葡萄品种值，并新增葡萄品种一列


,天数d,pH,可溶性固形物,总糖,总酸,百粒重,羟基肉桂酸_3,阿魏酸,白藜芦醇,表儿茶酸,...,儿茶酸,反式单阿魏酰酒石酸酯,根皮苷,槲皮苷,CXZ2,CXZ3,CXZ4,ML,MSL,感官得分
4,28,4.22,26.60,24.3,0.41,83.5,0.024,0.111,0.259,31.945,...,24.137,0.310,0.100,0.060,0,0,0,0,0,84.333333
5,33,4.15,26.80,23.8,0.34,101.0,0.018,0.072,0.219,39.131,...,29.351,0.293,0.072,0.026,0,0,0,0,0,84.500000
6,38,4.20,26.70,24.8,0.61,107.0,0.018,0.105,0.222,47.708,...,41.866,0.236,0.089,0.048,0,0,0,0,0,87.166667
7,43,4.21,29.00,22.5,0.64,102.0,0.014,0.115,0.125,32.611,...,22.658,0.266,0.026,0.038,0,0,0,0,0,81.666667
8,48,4.16,26.25,21.0,0.49,100.0,0.012,0.090,0.158,9.698,...,5.725,0.229,0.007,0.042,0,0,0,0,0,85.333333
13,28,4.07,27.20,25.1,0.39,87.0,0.060,0.128,0.893,49.712,...,37.553,1.226,0.303,0.119,0,0,0,0,1,79.000000
14,33,4.18,26.80,23.1,0.42,113.0,0.022,0.080,0.159,42.631,...,31.339,0.486,0.179,0.100,0,0,0,0,1,82.333333
15,38,4.08,27.90,25.4,0.61,114.0,0.023,0.110,0.325,39.915,...,34.521,0.504,0.167,0.062,0,0,0,0,1,86.333333
16,43,4.17,26.50,21.6,0.78,105.0,0.019,0.120,0.064,18.672,...,12.207,0.460,0.049,0.049,0,0,0,0,1,89.166667
17,48,4.14,27.88,22.2,0.64,102.0,0.019,0.161,0.204,28.822,...,19.338,0.410,0.051,0.032,0,0,0,0,1,83.333333


### 班级偏科学生查询（♥♥♥）

In [43]:
# 修改路径
file_score = 'A班C语言成绩统计new.xlsx'
df_stu_sco = pd.read_excel(file_score,sheet_name=None)

In [45]:
for i in df_stu_sco.keys():   # 读取全部sheep
    print(i)

成绩汇总
平时成绩
试题双向细目表
组卷详情
各题型得分
分题目得分


In [46]:
df_stu_sco_by  = df_stu_sco['成绩汇总']   # 取出 成绩汇总 赋值
df_stu_sco_by.drop('考勤分数',axis=1,inplace = True) #删除列，axis=1表示按列进行操作（axis=0则表示按行操作）
df_stu_sco_by

,编号,科目1,科目2,科目3,科目4,科目5,科目6,科目7,科目8
0,2,92.5,93.50,85.0,50.0,88.0,90.0,94,94
1,3,75.0,85.00,90.0,90.0,89.0,82.5,94,94
2,4,97.5,89.00,86.0,93.0,85.0,84.0,94,94
3,5,97.5,88.50,91.0,84.0,87.5,90.0,88,94
4,6,92.5,92.25,92.0,93.0,94.0,91.0,88,94
5,7,97.5,95.00,92.5,90.0,94.0,91.0,88,94
6,8,97.5,96.00,87.5,96.0,94.0,95.0,92,94
7,9,100.0,97.00,87.5,92.0,96.0,97.5,92,93
8,10,82.5,96.00,84.0,97.0,94.5,85.0,92,88
9,11,100.0,94.50,90.0,95.0,96.0,90.0,86,95


In [47]:
df_stu_sco_long = pd.melt(df_stu_sco_by,id_vars=['编号'])
# 将宽格式的数据转换为长格，编号列保持不变
#通过这个操作后，会转换为每个学生的每一项成绩占一行
df_stu_sco_long

,编号,variable,value
0,2,科目1,92.5
1,3,科目1,75.0
2,4,科目1,97.5
3,5,科目1,97.5
4,6,科目1,92.5
...,...,...,...
395,47,科目8,94.0
396,48,科目8,94.0
397,49,科目8,94.0
398,50,科目8,94.0


In [48]:
df_stu_sco_long = pd.melt(df_stu_sco_by,id_vars=['编号'],\
                          var_name = '科目名称',value_name = '科目成绩')
# pd.melt 函数将宽格式的数据转换为长格式，将多个列合并到新的列
df_stu_sco_long

,编号,科目名称,科目成绩
0,2,科目1,92.5
1,3,科目1,75.0
2,4,科目1,97.5
3,5,科目1,97.5
4,6,科目1,92.5
...,...,...,...
395,47,科目8,94.0
396,48,科目8,94.0
397,49,科目8,94.0
398,50,科目8,94.0


In [49]:
df_stu_sco_long['科目平均成绩'] = df_stu_sco_long.groupby('科目名称')['科目成绩'].transform('mean')
df_stu_sco_long['科目净成绩'] = df_stu_sco_long['科目成绩'] - df_stu_sco_long['科目平均成绩']
df_stu_sco_long

,编号,科目名称,科目成绩,科目平均成绩,科目净成绩
0,2,科目1,92.5,93.35,-0.85
1,3,科目1,75.0,93.35,-18.35
2,4,科目1,97.5,93.35,4.15
3,5,科目1,97.5,93.35,4.15
4,6,科目1,92.5,93.35,-0.85
...,...,...,...,...,...
395,47,科目8,94.0,92.88,1.12
396,48,科目8,94.0,92.88,1.12
397,49,科目8,94.0,92.88,1.12
398,50,科目8,94.0,92.88,1.12


In [52]:
df_stu_sco_long['科目平均成绩'] = df_stu_sco_long.groupby('科目名称')['科目成绩'].transform('mean') 
# 按照 “科目名称” 对数据进行分组,选取 “科目成绩” 这一列,对每个组应用均值函数
df_stu_sco_long['科目净成绩'] = df_stu_sco_long['科目成绩'] - df_stu_sco_long['科目平均成绩']
df_stu_sco_long

,编号,科目名称,科目成绩,科目平均成绩,科目净成绩
0,2,科目1,92.5,93.35,-0.85
1,3,科目1,75.0,93.35,-18.35
2,4,科目1,97.5,93.35,4.15
3,5,科目1,97.5,93.35,4.15
4,6,科目1,92.5,93.35,-0.85
...,...,...,...,...,...
395,47,科目8,94.0,92.88,1.12
396,48,科目8,94.0,92.88,1.12
397,49,科目8,94.0,92.88,1.12
398,50,科目8,94.0,92.88,1.12


In [53]:
df_stu_sco_long.groupby('编号')['科目净成绩'].agg(['max','min']).\
assign(diff = lambda x: x['max'] -x['min']).\
sort_values('diff',ascending=False)

,max,min,diff
编号,,,
2,8.06,-38.95,47.01
39,8.05,-20.85,28.90
37,6.65,-20.94,27.59
3,8.06,-18.35,26.41
36,5.31,-20.94,26.25
35,5.05,-20.94,25.99
38,4.32,-18.56,22.88
22,8.06,-13.95,22.01
40,2.06,-19.68,21.74


In [54]:
df_stu_sco_long.query('编号 == 2 ')

,编号,科目名称,科目成绩,科目平均成绩,科目净成绩
0,2,科目1,92.5,93.35,-0.85
50,2,科目2,93.5,89.68,3.82
100,2,科目3,85.0,88.40,-3.40
150,2,科目4,50.0,88.95,-38.95
200,2,科目5,88.0,87.19,0.81
250,2,科目6,90.0,88.56,1.44
300,2,科目7,94.0,85.94,8.06
350,2,科目8,94.0,92.88,1.12


In [55]:
df_stu_sco_long.query('编号 == 37 ')

,编号,科目名称,科目成绩,科目平均成绩,科目净成绩
35,37,科目1,100.00,93.35,6.65
85,37,科目2,69.75,89.68,-19.93
135,37,科目3,90.50,88.40,2.10
185,37,科目4,81.00,88.95,-7.95
235,37,科目5,92.50,87.19,5.31
285,37,科目6,95.00,88.56,6.44
335,37,科目7,65.00,85.94,-20.94
385,37,科目8,94.00,92.88,1.12


In [56]:
df_stu_sco_long.query('编号 == 34 ')


,编号,科目名称,科目成绩,科目平均成绩,科目净成绩
32,34,科目1,97.5,93.35,4.15
82,34,科目2,94.5,89.68,4.82
132,34,科目3,91.5,88.40,3.10
182,34,科目4,95.0,88.95,6.05
232,34,科目5,90.0,87.19,2.81
282,34,科目6,95.0,88.56,6.44
332,34,科目7,92.0,85.94,6.06
382,34,科目8,94.0,92.88,1.12


In [57]:
df_stu_sco_long.groupby('编号')['科目成绩'].std().sort_values()

编号
6      1.936203
34     2.389523
7      3.023716
8      3.105295
20     3.717118
49     3.853935
9      3.988824
42     4.061470
5      4.169939
28     4.223658
11     4.382901
26     4.397645
43     4.491560
30     4.499504
27     4.603958
31     4.860556
33     4.867898
46     4.894877
4      4.992405
16     5.159942
32     5.378529
23     5.394690
14     5.517727
48     5.665923
44     5.669073
21     5.688334
10     5.736786
25     5.743098
18     5.769377
12     5.890367
47     6.168758
45     6.226384
19     6.265606
29     6.352376
3      6.399986
13     6.450014
15     6.690879
51     6.959064
50     6.974443
17     7.115125
24     7.684560
41     7.905412
22     8.392107
40     8.489226
39     8.635217
38     8.867829
36     9.698122
35     9.807978
37    12.725423
2     14.848641
Name: 科目成绩, dtype: float64

In [58]:
df_stu_sco_long.groupby('编号')['科目成绩'].std().sort_values()\
.describe(percentiles=[0.01,0.025,0.05,0.25,0.75,0.95,0.975,0.99])

# groupby('编号')：按照“编号”进行分组，每个编号代表一个学生。
# ['科目成绩']：选择分组后的“科目成绩”列。
# .std()：计算每个学生所有科目成绩的标准差。
# 从小到大排序

count    50.000000
mean      6.014640
std       2.386506
min       1.936203
1%        2.158330
2.5%      2.532217
5%        3.060426
25%       4.493546
50%       5.678704
75%       6.892018
95%       9.758543
97.5%    12.068998
99%      13.808264
max      14.848641
Name: 科目成绩, dtype: float64